In [2]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [5]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [6]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
from tqdm.notebook import tqdm

In [7]:
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [8]:
split_txt_val_path = r'.\preprocessed_data\LJSpeech\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [9]:
len(val_uids)

512

# MCD

In [10]:
recon_folder = r'.\output\0629lj\result\LJSpeech\wav\reconstructed'
syn_folder = r'.\output\0629lj\result\LJSpeech\wav\synthesized'
gt_folder = r'.\Data\LJSpeech-1.1\wavs'

In [11]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_recon.append(result)

  0%|          | 0/512 [00:00<?, ?it/s]

CPU times: total: 6min 43s
Wall time: 8min 32s


In [12]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_gt.append(result)

  0%|          | 0/512 [00:00<?, ?it/s]

CPU times: total: 7min
Wall time: 8min 17s


In [13]:
lj_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [14]:
lj_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_LJ_new.csv")

In [15]:
print(f"MCDs_recon mean on LJSpeech: {np.nanmean(MCDs_recon)}")
print(f"MCDs_recon std on LJSpeech: {np.nanstd(MCDs_recon)}")

print(f"MCDs_gt mean on LJSpeech: {np.nanmean(MCDs_gt)}")
print(f"MCDs_gt std on LJSpeech: {np.nanstd(MCDs_gt)}")

MCDs_recon mean on LJSpeech: 5.215392116976927
MCDs_recon std on LJSpeech: 0.8072952880161278
MCDs_gt mean on LJSpeech: 6.059602374745914
MCDs_gt std on LJSpeech: 0.6979434652293569
